# DAT13 SF Homework 3 Solution

## Question 1

Use the bank.csv to explore the data. Observe the features: are they numbers? Are they strings? Are they binary? Are they continuous?

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
#import the data
df = pd.read_csv('../hw3/bank.csv', sep = ';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


Features

In [4]:
df.columns

Index([u'age', u'job', u'marital', u'education', u'default', u'balance',
       u'housing', u'loan', u'contact', u'day', u'month', u'duration',
       u'campaign', u'pdays', u'previous', u'poutcome', u'y'],
      dtype='object')

The description of the features is found on the bank-names.txt file:

```
Input variables:
   # bank client data:
   1 - age (numeric)
   2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
                                       "blue-collar","self-employed","retired","technician","services") 
   3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
   4 - education (categorical: "unknown","secondary","primary","tertiary")
   5 - default: has credit in default? (binary: "yes","no")
   6 - balance: average yearly balance, in euros (numeric) 
   7 - housing: has housing loan? (binary: "yes","no")
   8 - loan: has personal loan? (binary: "yes","no")
   # related with the last contact of the current campaign:
   9 - contact: contact communication type (categorical: "unknown","telephone","cellular") 
  10 - day: last contact day of the month (numeric)
  11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
  12 - duration: last contact duration, in seconds (numeric)
   # other attributes:
  13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
  14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
  15 - previous: number of contacts performed before this campaign and for this client (numeric)
  16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

  Output variable (desired target):
  17 - y - has the client subscribed a term deposit? (binary: "yes","no")
```


For later use I'll define lists of variables of the same kind

In [5]:
# Define lists of variables
categorical_variables = ['job', 'marital', 'education', 'contact', 'month', 'poutcome']
binary_variables = ['default', 'housing', 'loan', 'y']

string_variables = categorical_variables + binary_variables

## Question 2

Learn about label encoders here: http://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features and transform the features to numerical features

In [6]:
#encode features 
from sklearn import preprocessing

df_numerical = df.copy()
encoder_dict = {}

for col in string_variables:
    le = preprocessing.LabelEncoder()
    le.fit(df[col])
    encoder_dict[col] = le
    df_numerical[col] = le.transform(df[col])

df_numerical.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,10,1,0,0,1787,0,0,0,19,10,79,1,-1,0,3,0
1,33,7,1,1,0,4789,1,1,0,11,8,220,1,339,4,0,0
2,35,4,2,2,0,1350,1,0,0,16,0,185,1,330,1,0,0
3,30,4,1,2,0,1476,1,1,2,3,6,199,4,-1,0,3,0
4,59,1,1,1,0,0,1,0,2,5,8,226,1,-1,0,3,0


In [49]:
encoder_dict

{'contact': LabelEncoder(),
 'default': LabelEncoder(),
 'education': LabelEncoder(),
 'housing': LabelEncoder(),
 'job': LabelEncoder(),
 'loan': LabelEncoder(),
 'marital': LabelEncoder(),
 'month': LabelEncoder(),
 'poutcome': LabelEncoder(),
 'y': LabelEncoder()}

In [25]:
encoder_dict['poutcome'].classes_

array(['failure', 'other', 'success', 'unknown'], dtype=object)

## Question 3

Build a simple decision tree model to predict the classification goal.

In [50]:
# imports
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier


from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

In [27]:
xcols = [col for col in df_numerical.columns if col != 'y']

X = df_numerical[xcols].values
y = df_numerical['y'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [52]:
clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


conf_mat = confusion_matrix(y_test, y_pred)

print(conf_mat )

[[920  73]
 [ 79  59]]


In [29]:
#with open("bank.dot", 'wb') as f:
#          f = export_graphviz(clf, out_file=f)

In [30]:
#! dot -Tpng bank.dot -o bank.png

/bin/sh: dot: command not found


In [31]:
#from IPython.display import Image
 
#Image("bank.png")

## Question 4

Evaluate the result of the classification with cross-validation.

In [53]:
from sklearn import cross_validation

In [54]:
scores = cross_validation.cross_val_score(clf, X, y, cv=cross_validation.KFold(len(X), n_folds=5,
                           shuffle=True, random_state=0))

scores

array([ 0.84183054,  0.83770333,  0.83126973,  0.84084011,  0.83926187])

In [55]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.84 (+/- 0.01)


## Question 5

Extend the analysis and cross-validation to bank-additional-full.csv. How does the performance change?

In [57]:
df_additional_full = pd.read_csv('homework/Hw3/bank-additional-full.csv', sep = ';')


```
   1 - age (numeric)
   2 - job : type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")
   3 - marital : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)
   4 - education (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")
   5 - default: has credit in default? (categorical: "no","yes","unknown")
   6 - housing: has housing loan? (categorical: "no","yes","unknown")
   7 - loan: has personal loan? (categorical: "no","yes","unknown")
   # related with the last contact of the current campaign:
   8 - contact: contact communication type (categorical: "cellular","telephone") 
   9 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
  10 - day_of_week: last contact day of the week (categorical: "mon","tue","wed","thu","fri")
  11 - duration: last contact duration, in seconds (numeric). Important note:  this attribute highly affects the output target (e.g., if duration=0 then y="no"). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
   # other attributes:
  12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
  13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
  14 - previous: number of contacts performed before this campaign and for this client (numeric)
  15 - poutcome: outcome of the previous marketing campaign (categorical: "failure","nonexistent","success")
   # social and economic context attributes
  16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
  17 - cons.price.idx: consumer price index - monthly indicator (numeric)     
  18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)     
  19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
  20 - nr.employed: number of employees - quarterly indicator (numeric)

  Output variable (desired target):
  21 - y - has the client subscribed a term deposit? (binary: "yes","no")
```

In [58]:
full_categorical_variables = ['job', 'marital', 'education', 'contact', 'month', 'day_of_week', 'poutcome']
full_binary_variables = ['default', 'housing', 'loan', 'y']

full_string_variables = full_categorical_variables + full_binary_variables


df_additional_full_numerical = df_additional_full.copy()
additional_full_encoder_dict = {}

for col in full_string_variables:
    le = preprocessing.LabelEncoder()
    le.fit(df_additional_full[col])
    additional_full_encoder_dict[col] = le
    df_additional_full_numerical[col] = le.transform(df_additional_full[col])

df_additional_full_numerical.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,1,0,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191,0
1,57,7,1,3,1,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191,0
2,37,7,1,3,0,2,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191,0
3,40,0,1,1,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191,0
4,56,7,1,3,0,0,2,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191,0


In [59]:
df_additional_full.columns

Index([u'age', u'job', u'marital', u'education', u'default', u'housing', u'loan', u'contact', u'month', u'day_of_week', u'duration', u'campaign', u'pdays', u'previous', u'poutcome', u'emp.var.rate', u'cons.price.idx', u'cons.conf.idx', u'euribor3m', u'nr.employed', u'y'], dtype='object')

In [60]:
xcols = [col for col in df_additional_full_numerical.columns if col not in ['y', 'duration']]

X = df_additional_full_numerical[xcols].values
y = df_additional_full_numerical['y'].values

In [61]:
scores = cross_validation.cross_val_score(clf, X, y, cv=cross_validation.KFold(len(X), n_folds=5,
                           shuffle=True, random_state=0))
scores

array([ 0.84207332,  0.84085943,  0.83284778,  0.83537696,  0.84084011])

In [62]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.84 (+/- 0.01)
